In [14]:
import pandas as pd

economic_data = pd.read_csv('/workspaces/Electoral-Economic-Analysis/electoral_analysis/processed_bea_economic_growth.csv')
electoral_data = pd.read_csv('/workspaces/Electoral-Economic-Analysis/electoral_analysis/processed_electoral_data.csv')

states = ["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware", "Florida", "Georgia", 
          "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts", 
          "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", "New Jersey", 
          "New Mexico", "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", 
          "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", "West Virginia", 
          "Wisconsin", "Wyoming", "District of Columbia"]

economic_data = economic_data[economic_data['State'].isin(states)]


# print(economic_data.head())

# print(economic_data)

In [15]:
# Electoral data filtering

election_2008_data = electoral_data[electoral_data['Year'] == 2008]
election_2012_data = electoral_data[electoral_data['Year'] == 2012]
election_2016_data = electoral_data[electoral_data['Year'] == 2016]
election_2020_data = electoral_data[electoral_data['Year'] == 2020]

# Print the number of unique electoral result in each filtered dataset
print("Unique electoral results in 2008:", election_2008_data['State'].nunique())
print("Unique electoral results in 2012:", election_2012_data['State'].nunique())
print("Unique electoral results in 2016:", election_2016_data['State'].nunique())
print("Unique electoral results in 2020:", election_2020_data['State'].nunique())

Unique electoral results in 2008: 51
Unique electoral results in 2012: 51
Unique electoral results in 2016: 51
Unique electoral results in 2020: 51


In [16]:
#cycle_duration = ['2009-2013' '2013-2017' '2017-2021' '2021-2023']

# Filter data for Real GDP
realGDP_data_2009_2013 = economic_data[(economic_data['Indicator'] == 'Real GDP') & (economic_data['Cycle Duration'] == '2009-2013')]
realGDP_data_2013_2017 = economic_data[(economic_data['Indicator'] == 'Real GDP') & (economic_data['Cycle Duration'] == '2013-2017')]
realGDP_data_2017_2021 = economic_data[(economic_data['Indicator'] == 'Real GDP') & (economic_data['Cycle Duration'] == '2017-2021')]
realGDP_data_2021_2023 = economic_data[(economic_data['Indicator'] == 'Real GDP') & (economic_data['Cycle Duration'] == '2021-2023')]

# Filter data for Real per capita personal income
real_per_capita_personal_income_data_2009_2013 = economic_data[(economic_data['Indicator'] == 'Real Per Capita Personal Income') & (economic_data['Cycle Duration'] == '2009-2013')]
real_per_capita_personal_income_data_2013_2017 = economic_data[(economic_data['Indicator'] == 'Real Per Capita Personal Income') & (economic_data['Cycle Duration'] == '2013-2017')]
real_per_capita_personal_income_data_2017_2021 = economic_data[(economic_data['Indicator'] == 'Real Per Capita Personal Income') & (economic_data['Cycle Duration'] == '2017-2021')]
real_per_capita_personal_income_data_2021_2023 = economic_data[(economic_data['Indicator'] == 'Real Per Capita Personal Income') & (economic_data['Cycle Duration'] == '2021-2023')]

# Filter data for Real personal income
real_personal_income_data_2009_2013 = economic_data[(economic_data['Indicator'] == 'Real Personal Income') & (economic_data['Cycle Duration'] == '2009-2013')]
real_personal_income_data_2013_2017 = economic_data[(economic_data['Indicator'] == 'Real Personal Income') & (economic_data['Cycle Duration'] == '2013-2017')]
real_personal_income_data_2017_2021 = economic_data[(economic_data['Indicator'] == 'Real Personal Income') & (economic_data['Cycle Duration'] == '2017-2021')]
real_personal_income_data_2021_2023 = economic_data[(economic_data['Indicator'] == 'Real Personal Income') & (economic_data['Cycle Duration'] == '2021-2023')]

# Filter data for Real per capita PCE
real_per_capita_PCE_data_2009_2013 = economic_data[(economic_data['Indicator'] == 'Real Per Capita PCE') & (economic_data['Cycle Duration'] == '2009-2013')]
real_per_capita_PCE_data_2013_2017 = economic_data[(economic_data['Indicator'] == 'Real Per Capita PCE') & (economic_data['Cycle Duration'] == '2013-2017')]
real_per_capita_PCE_data_2017_2021 = economic_data[(economic_data['Indicator'] == 'Real Per Capita PCE') & (economic_data['Cycle Duration'] == '2017-2021')]
real_per_capita_PCE_data_2021_2023 = economic_data[(economic_data['Indicator'] == 'Real Per Capita PCE') & (economic_data['Cycle Duration'] == '2021-2023')]

# Filter data for Real PCE
real_PCE_data_2009_2013 = economic_data[(economic_data['Indicator'] == 'Real PCE') & (economic_data['Cycle Duration'] == '2009-2013')]
real_PCE_data_2013_2017 = economic_data[(economic_data['Indicator'] == 'Real PCE') & (economic_data['Cycle Duration'] == '2013-2017')]
real_PCE_data_2017_2021 = economic_data[(economic_data['Indicator'] == 'Real PCE') & (economic_data['Cycle Duration'] == '2017-2021')]
real_PCE_data_2021_2023 = economic_data[(economic_data['Indicator'] == 'Real PCE') & (economic_data['Cycle Duration'] == '2021-2023')]

# Filter data for Total employment
total_employment_data_2009_2013 = economic_data[(economic_data['Indicator'] == 'Total Employment') & (economic_data['Cycle Duration'] == '2009-2013')]
total_employment_data_2013_2017 = economic_data[(economic_data['Indicator'] == 'Total Employment') & (economic_data['Cycle Duration'] == '2013-2017')]
total_employment_data_2017_2021 = economic_data[(economic_data['Indicator'] == 'Total Employment') & (economic_data['Cycle Duration'] == '2017-2021')]
total_employment_data_2021_2023 = economic_data[(economic_data['Indicator'] == 'Total Employment') & (economic_data['Cycle Duration'] == '2021-2023')]

# Print the number of unique states in each filtered dataset
print(len(realGDP_data_2009_2013['State'].unique()))
print(len(real_per_capita_personal_income_data_2009_2013['State'].unique()))
print(len(real_personal_income_data_2009_2013['State'].unique()))
print(len(real_per_capita_PCE_data_2009_2013['State'].unique()))
print(len(real_PCE_data_2009_2013['State'].unique()))
print(len(total_employment_data_2009_2013['State'].unique()))

# Print the number of unique cycle durations in each filtered dataset
print(len(realGDP_data_2009_2013['Cycle Duration'].unique()))
print(len(real_per_capita_personal_income_data_2009_2013['Cycle Duration'].unique()))
print(len(real_personal_income_data_2009_2013['Cycle Duration'].unique()))
print(len(real_per_capita_PCE_data_2009_2013['Cycle Duration'].unique()))
print(len(real_PCE_data_2009_2013['Cycle Duration'].unique()))
print(len(total_employment_data_2009_2013['Cycle Duration'].unique()))

# print(realGDP_data_2009_2013.head())


51
51
51
51
51
51
1
1
1
1
1
1


In [ ]:
def load_and_merge_economic_electoral_data(economic_data_path, electoral_data_path, indicator, cycle_duration, election_year):
    # Load the datasets
    economic_data = pd.read_csv(economic_data_path)
    electoral_data = pd.read_csv(electoral_data_path)

    # Normalize the column names for merging
    economic_data.rename(columns={"State": "state", "Cycle Duration": "cycle_duration"}, inplace=True)
    electoral_data.rename(columns={"State": "state", "Year": "year"}, inplace=True)

    # Ensure 'state' columns are in the same format (e.g., uppercase)
    economic_data['state'] = economic_data['state'].str.upper()
    electoral_data['state'] = electoral_data['state'].str.upper()

    # Ensure 'year' columns are integers
    electoral_data['year'] = electoral_data['year'].astype(int)

    # Filter economic data for the specified indicator and cycle duration
    economic_df = economic_data[(economic_data['Indicator'] == indicator) & (economic_data['cycle_duration'] == cycle_duration)]

    # Adjust the end year to match the election year
    economic_df['End Year'] = economic_df['cycle_duration'].str.split('-').str[1].astype(int) - 1

    # Filter the economic data to match the election year
    economic_df = economic_df[economic_df['End Year'] == election_year]

    # Filter electoral data for the specified election year
    election_df = electoral_data[electoral_data['year'] == election_year]

    # Merge the datasets
    merged_df = pd.merge(economic_df, election_df, left_on=['state', 'End Year'], right_on=['state', 'year'], how='inner')

    # Drop unnecessary columns
    merged_df = merged_df.drop(columns=["End Year"])

    return merged_df

In [17]:
# Assuming real_personal_income_data_2009_2013 and election_2012_data are already defined

# Normalize the column names for merging
real_personal_income_data_2009_2013.rename(columns={"State": "state", "Cycle Duration": "cycle_duration"}, inplace=True)
election_2012_data.rename(columns={"State": "state", "Year": "year"}, inplace=True)

# Step 1: Extract the end year from the Cycle Duration and adjust it to match the election year
real_personal_income_data_2009_2013['cycle_duration'] = real_personal_income_data_2009_2013['cycle_duration'].astype(str)
real_personal_income_data_2009_2013['End Year'] = real_personal_income_data_2009_2013['cycle_duration'].str.split('-').str[1].astype(int) - 1

# Ensure 'state' columns are in the same format (e.g., uppercase)
real_personal_income_data_2009_2013['state'] = real_personal_income_data_2009_2013['state'].str.upper()
election_2012_data['state'] = election_2012_data['state'].str.upper()

# Ensure 'year' columns are integers
real_personal_income_data_2009_2013['End Year'] = real_personal_income_data_2009_2013['End Year'].astype(int)
election_2012_data['year'] = election_2012_data['year'].astype(int)

# Merging two relative dataframes
merged_data = pd.merge(real_personal_income_data_2009_2013, election_2012_data, left_on=['state', 'End Year'], right_on=['state', 'year'], how='inner')

# Drop unnecessary columns and clean up the data
merged_data = merged_data.drop(columns=["End Year"])

# Display the merged dataset
print(merged_data.head())

# Save the merged dataset to a CSV file
merged_data.to_csv('/workspaces/Electoral-Economic-Analysis/electoral_analysis/merged_real_personal_income_2012.csv', index=False)

        state             Indicator     Term     President cycle_duration  \
0     ALABAMA  Real Personal Income  Obama 1  Barack Obama      2009-2013   
1      ALASKA  Real Personal Income  Obama 1  Barack Obama      2009-2013   
2     ARIZONA  Real Personal Income  Obama 1  Barack Obama      2009-2013   
3    ARKANSAS  Real Personal Income  Obama 1  Barack Obama      2009-2013   
4  CALIFORNIA  Real Personal Income  Obama 1  Barack Obama      2009-2013   

   Overall Growth (%)  Average Growth (%)  year        Office  \
0                0.68                0.17  2012  US PRESIDENT   
1               12.92                3.15  2012  US PRESIDENT   
2               10.61                2.56  2012  US PRESIDENT   
3                5.57                1.40  2012  US PRESIDENT   
4               11.99                2.88  2012  US PRESIDENT   

  Winning Candidate Winning Party  Winning Percentage  Losing Candidate  \
0      ROMNEY, MITT    REPUBLICAN               60.55  OBAMA, BARACK H.

/tmp/ipykernel_18425/3128563738.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  real_personal_income_data_2009_2013.rename(columns={"State": "state", "Cycle Duration": "cycle_duration"}, inplace=True)
/tmp/ipykernel_18425/3128563738.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  election_2012_data.rename(columns={"State": "state", "Year": "year"}, inplace=True)
/tmp/ipykernel_18425/3128563738.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa